In [1]:
!mkdir data

In [2]:
%%capture
!pip install gdown

In [3]:
!cd data && gdown "10nb-yVKttNsZ-UUqlFxxZ8pZfZo2Bjkz" && gdown "19gqhWlyEKo2d_j3dk7Hxz4LX0M6do0s_"

Downloading...
From: https://drive.google.com/uc?id=10nb-yVKttNsZ-UUqlFxxZ8pZfZo2Bjkz
To: /kaggle/working/data/train.jsonl
100%|███████████████████████████████████████| 4.87M/4.87M [00:00<00:00, 185MB/s]
Downloading...
From: https://drive.google.com/uc?id=19gqhWlyEKo2d_j3dk7Hxz4LX0M6do0s_
To: /kaggle/working/data/test.jsonl
100%|█████████████████████████████████████████| 507k/507k [00:00<00:00, 105MB/s]


In [4]:
!pip install -q peft transformers datasets evaluate seqeval accelerate bitsandbytes trl

In [5]:
from tokenizers import BertWordPieceTokenizer, ByteLevelBPETokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification, AutoTokenizer
from transformers import pipeline
from peft import IA3Config, TaskType, get_peft_model, LoraConfig
import json
import random
import pandas as pd
import numpy as np
import evaluate
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

import os
import warnings

warnings.filterwarnings('ignore')

np.random.seed(0)
torch.manual_seed(0)
random.seed(0)

2024-04-26 17:56:01.333981: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 17:56:01.334133: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 17:56:01.477560: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
model_name =  'ai-forever/ruRoberta-large' # 'DeepPavlov/rubert-base-cased'
wp_tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

In [7]:
wp_tokenizer.encode_plus("Hello_there", return_token_type_ids=True, return_special_tokens_mask=True, return_offsets_mapping=True)

{'input_ids': [1, 44, 9691, 83, 67, 7889, 10524, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1], 'special_tokens_mask': [1, 0, 0, 0, 0, 0, 0, 1], 'offset_mapping': [(0, 0), (0, 1), (1, 4), (4, 5), (5, 6), (6, 8), (8, 11), (0, 0)]}

In [8]:
NERS = ['AGE', 'AWARD', 'CITY', 'COUNTRY', 'CRIME', 'DATE', 'DISEASE', 'DISTRICT', 'EVENT', 'FACILITY', 'FAMILY', 
        'IDEOLOGY', 'LANGUAGE', 'LAW', 'LOCATION', 'MONEY', 'NATIONALITY', 'NUMBER', 'ORDINAL', 'ORGANIZATION', 
        'PENALTY', 'PERCENT', 'PERSON', 'PRODUCT', 'PROFESSION', 'RELIGION', 'STATE_OR_PROVINCE', 'TIME', 'WORK_OF_ART'
]

In [9]:
# Code adapted from https://github.com/dialogue-evaluation/RuNNE/blob/main/baseline/iobes_flat_dataset.py
class IOBESFlatRuNNEDataset(Dataset):

    def __init__(self, dataset, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.all_data = dataset

        # Filter entities, leave lowest-level ones
        flat_data = []
        for c in self.all_data:
            flats = []
            for ec in c["entities"]:
                start = ec["start"]
                end = ec["end"]
                # Lowest-level entities
                if len([e for e in c["entities"] if e["start"] >= start and e["end"] < end or e["start"] > start and e["end"] <= end]) == 0:
                    flats.append(ec)
            flat_data.append({**c, "entities" : flats})
        self.flat_data = flat_data
        
        # Convert entites bound to token-wise labels
        new_data = []
        for c in self.flat_data:
            context = c["context"]
            encodings = self.tokenizer.encode_plus(context, return_token_type_ids=True, return_special_tokens_mask=True, return_offsets_mapping=True)
            tokens = encodings['input_ids']
            offsets = encodings['offset_mapping']
            attention_mask = encodings['attention_mask']

            origin_offset2token_idx_start = {}
            origin_offset2token_idx_end = {}
            for token_idx in range(len(tokens)):
                token_start, token_end = offsets[token_idx]
                # skip [CLS] or [SEP]
                if token_start == token_end == 0:
                    continue

                token_end -= 1
                origin_offset2token_idx_start[token_start] = token_idx
                origin_offset2token_idx_end[token_end] = token_idx
            
            valid_entities = []
            # Convert char indices for entities into token indices
            for e in c["entities"]:
                start = e["start"]
                end = e["end"]
                try:
                    new_start = origin_offset2token_idx_start[start]
                    new_end = origin_offset2token_idx_end[end]
                except KeyError:
                    pass

                valid_entities.append({
                        **e,
                        "tl_start" : new_start,
                        "tl_end" : new_end
                    })
            new_data.append({
                    **c,
                    "entities" : valid_entities,
                    "tokens" : tokens,
                    "attention_mask" : attention_mask,
                    "offsets" : offsets,
                })

        self.flat_data = new_data

        labeled_data = []
        # IOBES labeling
        for c in self.flat_data:
            labels = []
            for e in c["entities"]:
                labels.append((e["tag"], e["tl_start"], e["tl_end"]))
            label_seq = ["O"] * len(c["tokens"])

            for tag, start, end in labels:
                if start == end: # S
                    label_seq[start] = 'S-' + tag
                else: # BI*E
                    label_seq[start] = 'B-' + tag 
                    label_seq[end] = 'E-' + tag 
                    for i in range(start + 1, end):
                        label_seq[i] = 'I-' + tag

            labeled_data.append({
                    **c,
                    "labels" : label_seq
                })

        self.data = labeled_data

        tags = [t.strip() for t in NERS]

        tag_to_id = {}
        # IOBES labeling
        for idx, tag in enumerate(tags):
            tag_to_id['B-' + tag] = idx * 4 + 1
            tag_to_id['I-' + tag] = idx * 4 + 2
            tag_to_id['E-' + tag] = idx * 4 + 3
            tag_to_id['S-' + tag] = idx * 4 + 4
        tag_to_id['O'] = 0

        self.tag_to_id = tag_to_id

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        data = self.data[item]

        assert len(data["tokens"]) == len(data["attention_mask"])
        assert len(data["attention_mask"]) == len(data["labels"])

        labels_ids = [self.tag_to_id[tag] for tag in data["labels"]]

        tokens = data["tokens"]
        attention_mask = data["attention_mask"]
        offsets = data["offsets"]

        # truncate to max_len tokens
        tokens = tokens[: self.max_length]
        attention_mask = attention_mask[: self.max_length]
        labels_ids = labels_ids[: self.max_length]
        offsets = offsets[: self.max_length]

        sep_token = self.tokenizer.sep_token_id
        if tokens[-1] != sep_token:
            assert len(tokens) == self.max_length
            tokens = tokens[: -1] + [sep_token]

        return {'input_ids': torch.LongTensor(tokens),
                'attention_mask': torch.LongTensor(attention_mask),
                'labels': torch.LongTensor(labels_ids)}

# Prepare train/val/test data

In [10]:
train_data = pd.read_json('data/train.jsonl', lines=True).drop(["id"], axis=1)
runne_dataset = load_dataset("iluvvatar/RuNNE", trust_remote_code=True)['train'].to_pandas().drop(['id'], axis=1)
runne_dataset['entities'] = runne_dataset['entities'].apply(lambda x: [[int(ent.split()[0]),
                                                                        int(ent.split()[1]) - 1,
                                                                        ent.split()[2]] for ent in x])
train_data = pd.concat([runne_dataset.rename(columns={'text': 'sentences', 'entities': 'ners'}),
                        train_data], ignore_index=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

In [11]:
len(train_data), len(runne_dataset), len(pd.read_json('data/train.jsonl', lines=True))

(980, 461, 519)

In [12]:
train_data = train_data.drop_duplicates(subset='sentences')
len(train_data)

796

In [13]:
train_data, val_data = train_test_split(train_data, train_size=0.8, random_state=0)

In [14]:
# List of dict{'entities': {'tag', 'start', 'end'}, 'context'}
train_dataset = []
for ners, sentence in zip(train_data.ners, train_data.sentences):
    train_dataset.append({
        'entities': [{'tag': x[2], 'start': x[0], 'end': x[1]} for x in ners],
        'context': sentence
    })

val_dataset = []
for ners, sentence in zip(val_data.ners, val_data.sentences):
    val_dataset.append({
        'entities': [{'tag': x[2], 'start': x[0], 'end': x[1]} for x in ners],
        'context': sentence
    })

In [15]:
flat_train_dataset = IOBESFlatRuNNEDataset(dataset = train_dataset, 
                                     tokenizer = wp_tokenizer,
                                     max_length = 512)

flat_val_dataset = IOBESFlatRuNNEDataset(dataset = val_dataset, 
                                     tokenizer = wp_tokenizer,
                                     max_length = 512)

In [16]:
data_collator = DataCollatorForTokenClassification(wp_tokenizer)

# Prepare train

In [17]:
seqeval = evaluate.load("seqeval")

In [18]:
label_list = list(flat_train_dataset.tag_to_id.items())
label_list.sort(key=lambda x: x[1])
label_list = [label[0] for label in label_list]

In [19]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [20]:
label2id = flat_train_dataset.tag_to_id
id2label = {i: label for label, i in label2id.items()}

In [21]:
model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=29 * 4 + 1, id2label=id2label, label2id=label2id
)

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
peft_config = IA3Config(
    task_type=TaskType.TOKEN_CLS, target_modules=["query", "key", "output.dense"], feedforward_modules=["output.dense"]
)
peft_config = LoraConfig(task_type=TaskType.TOKEN_CLS, r=8)

In [23]:
model = get_peft_model(model, peft_config)

In [24]:
training_args = TrainingArguments(
    output_dir="checkpoints",
    learning_rate=0.001,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=flat_train_dataset,
    eval_dataset=flat_val_dataset,
    tokenizer=wp_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [25]:
import os

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb_api")

os.environ['WANDB_API_KEY'] = wandb_api

In [26]:
trainer.train()

wandb: Currently logged in as: k-galliamov. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.527059,0.568768,0.649437,0.606431,0.860717
2,No log,0.332797,0.684829,0.770867,0.725305,0.899401
3,No log,0.287917,0.715113,0.809200,0.759253,0.911802
4,No log,0.274959,0.756476,0.801657,0.778411,0.917416
5,No log,0.255764,0.771161,0.813404,0.791719,0.923373
6,No log,0.250172,0.761093,0.844194,0.800492,0.923680
7,No log,0.244034,0.770143,0.850995,0.808553,0.924997
8,No log,0.234873,0.774736,0.853963,0.812423,0.927633
9,No log,0.226635,0.790562,0.855571,0.821783,0.932398
10,No log,0.223449,0.807733,0.852479,0.829503,0.934817


TrainOutput(global_step=800, training_loss=0.2501101016998291, metrics={'train_runtime': 1820.1361, 'train_samples_per_second': 6.988, 'train_steps_per_second': 0.44, 'total_flos': 1.1352646610408208e+16, 'train_loss': 0.2501101016998291, 'epoch': 20.0})

In [27]:
classifier = pipeline("ner", model=model.merge_and_unload(), tokenizer=wp_tokenizer)

# Making the prediction

In [28]:
dev_set = pd.read_json('data/test.jsonl', lines=True)
dev_set

,senences,id
0,Владелец «Бирмингема» получил шесть лет тюрьмы...,584
1,Акция протеста на Майдане Независимости объявл...,585
2,Фольксваген может перейти под контроль Порше \...,586
3,В Москве покажут фильмы Чарли Чаплина с живой ...,587
4,Чулпан Хаматова сыграет главную роль в фильме ...,588
...,...,...
60,ОБСЕ назвала референдум о статусе Крыма незако...,644
61,Египетского студента могут выслать из страны з...,645
62,Геннадий Онищенко отправлен в отставку\nГеннад...,646
63,Племянник Алишера Усманова разбился в ДТП\nВид...,647


In [29]:
!rm -rf data
!rm -rf wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
results = []
aggregation_strategy = 'average' # simple

for sent in tqdm(dev_set.senences):
    try:
        tags = classifier(sent, aggregation_strategy=aggregation_strategy) # simple
        tags = [(tag['start'], tag['end'] - 1, tag['entity_group']) for tag in tags]
    except RuntimeError:
        subsents = sent.split('\n')
        tags = []
        offset = 0

        for subsent in subsents:
            if subsent == '':
                offset += 1
                continue
            subtags = classifier(subsent, aggregation_strategy=aggregation_strategy)
            subtags = [(tag['start'] + offset, tag['end'] + offset - 1, tag['entity_group']) for tag in subtags]
            tags.extend(subtags)
            offset += len(subsent) + 1

    tags.sort()
    new_tags = []
    for tag in tags:
        if len(new_tags) != 0 and tag[0] - new_tags[-1][1] <= 2 and new_tags[-1][2] == tag[2]:
            new_tags[-1] = (new_tags[-1][0], tag[1], tag[2])
        else:
            new_tags.append(tag)
    tags = new_tags

    results.append(tags)

  0%|          | 0/65 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [33]:
output_file = "test.jsonl"

# Check if the file exists and delete it
if os.path.exists(output_file):
    os.remove(output_file)

for tags, _id in zip(results, dev_set['id']):
    ner_list = [[start, end, tag] for start, end, tag in tags]
    json_obj = {"ners": ner_list, "id": _id}

    with open(output_file, "a") as f:
        f.write(json.dumps(json_obj) + "\n")

In [34]:
!ls

checkpoints  test.jsonl  test_merge.jsonl  wandb
